# Introduction: Partition Labeling

In this notebook, we will work with a single partition to develop a labeling function. The end goal is code that can take a partition on disk and generate labels with the cutoff time dataframe. This will then be parallelized using Dask.

In [1]:
import pandas as pd 
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
PARTITION = '100'
base_dir = '/data/churn/partitions/'
directory = base_dir + 'p' + PARTITION

import os
os.listdir(directory)

['logs.csv',
 'members.csv',
 'train.csv',
 'cancel_cutoff_times.csv',
 'test.csv',
 'month_labels.csv',
 'transactions.csv',
 'cutoff_times.csv']

In [3]:
all_partitions = os.listdir('/data/churn/partitions/')
len(all_partitions)

1000

### Read in Data Files

The only data we need for making the labels is the transactions. However, we'll read in all the data here.

In [4]:
members = pd.read_csv(f'{directory}/members.csv', 
                      parse_dates=['registration_init_time'], infer_datetime_format = True)
trans = pd.read_csv(f'{directory}/transactions.csv',
                   parse_dates=['transaction_date', 'membership_expire_date'], infer_datetime_format = True)
logs = pd.read_csv(f'{directory}/logs.csv', parse_dates = ['date'])

In [5]:
members.head()
trans.head()
logs.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,46r0VsNbxboshxpjyJ91oWdGX8SHzxFuS2mWkEBujD4=,13,29,female,9,2007-04-29
1,0C2o0WrDEiGkjbQtOR8x3U05OVCVLYFKHVRjgPCA0mM=,10,20,male,9,2007-11-07
2,d1UX5bu9bMtb8mzId7VbHFIUa46UO+IElVh9CxbonNQ=,1,0,NaN,9,2015-01-28
3,mF6w9kCGjtrI0PtOomXQTmN027pGL21K8E2Jvitb0RE=,1,0,NaN,9,2016-12-31
4,EjxD7eoFZ1+/jsVZc+8J79lRbHpZK+ZtyRGZFtZ0at4=,1,0,NaN,4,2017-01-17


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,G7TmHc9Gg2t8ovG/KFaB53We/0CQPELhZ5UUN2Ol3AQ=,39,30,149,149,1,2015-09-30,2015-11-13,0
1,LPbp8N7VRuqEISEVim8ppTaeYJG/rWS/t4g/dEFuWjw=,34,30,149,149,1,2016-02-29,2016-03-31,0
2,xvYqULBWzJvN8heyFtY3hbY3egyQNbXuDx0igtsoi00=,29,30,180,180,1,2017-01-31,2017-03-01,0
3,UR4iin4mAkajoa7o+AyTTmz5k3N2GR3/rZY8a4KwADI=,41,30,99,99,1,2017-01-31,2017-02-28,0
4,ax8CRhY8BMRA/ZvT1wI+2N/EdPXiSPGxa9y7bntA1Uc=,40,30,149,149,1,2016-05-04,2016-06-08,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,zraYK27dg2odAmzM1z3r4DKqx/9X6P/O7f3VQbTF3zU=,2017-03-18,9,0,1,0,22,26,5714.520
1,Cv+9XMTbZ6Ua6TZoJSJJl7gtkstlXa3R/LHl2zdK+PY=,2017-03-09,4,0,0,0,28,30,3701.484
2,6bL/KHOSEXgB9yeqNpwAZDc9LYd7I/JbUUpXpTAyyKI=,2017-03-21,1,2,1,2,15,16,4288.633
3,UL9bm0eoKgK6YQWCGwz8CtD/9ySFy1fbFBzqUF2jmEs=,2017-03-30,13,3,0,0,30,37,8551.879
4,6cPyxd/lWz24stfnNlsWbW839l5GVWCQ/oJzr7ZT4a4=,2017-03-24,45,9,6,3,32,81,11342.642


## Remove Anomalies

There are a number of records in the transactinos where the `membership_expire_date` is before the `transaction_date`. We will remove these to prevent them from creating issues with our labeling.

In [6]:
trans = trans.loc[trans['membership_expire_date'] >= trans['transaction_date']]

Some of the transactions occur on the same day for the same customer. In these cases, we will assume that the transaction with the later `membership_expire_date` happened second. 

In [7]:
trans = trans.sort_values(['msno', 'transaction_date', 'membership_expire_date'])

# View value counts of customers
trans.groupby('msno').count().sort_values('is_cancel').tail()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
msno,,,,,,,,
5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,39,39,39,39,39,39,39,39
CNVTd5cNoA6WRxMf7VveVwYyrOoDgFs5xmm1122qQNc=,39,39,39,39,39,39,39,39
kA7JMc6Q6nAFw1fgcs0hejCw4xXaGUzB/eq+M5n21wQ=,40,40,40,40,40,40,40,40
mFsXs71TCuJOnHKWQQ271BdecYkrXgPIRU7VUfmtqAY=,42,42,42,42,42,42,42,42
/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,51,51,51,51,51,51,51,51


We'll work with the data from one customer at first. 

In [8]:
cust = trans.loc[trans['msno'] == '/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM='].copy()
cust.iloc[:, 1:].tail(6)

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
12519,41,30,129,129,1,2016-11-08,2016-12-08,0
7776,41,30,129,129,1,2016-11-14,2016-11-14,1
1504,41,30,99,99,1,2016-11-14,2016-12-13,0
5462,41,30,99,99,1,2016-12-13,2017-01-13,0
10717,41,30,99,99,1,2017-01-13,2017-02-13,0
21899,41,30,99,99,1,2017-03-13,2017-04-13,0


In [9]:
cancel = cust.loc[cust['is_cancel'] == 1].copy()
cancel

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
7776,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,129,129,1,2016-11-14,2016-11-14,1


## Find Churns

To find a churn, we'll set a variable `periods` that defines the number of days required without a membership before a customer is considered churned. Then, if there are any gaps in membership longer than this period, we will mark it as a churn. To find the time period without a membership, we'll look at each `membership_expire_date` and calculate the time to the next transaction that is not a cancel. If this value is greater than `periods`, then the customer has churned.

In [10]:
cust.reset_index(inplace = True, drop = True)
periods = 30

In [11]:
for tup in cust.itertuples():
    break
tup

Pandas(Index=0, msno='/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=', payment_method_id=41, payment_plan_days=30, plan_list_price=149, actual_amount_paid=149, is_auto_renew=1, transaction_date=Timestamp('2015-01-13 00:00:00'), membership_expire_date=Timestamp('2016-12-15 00:00:00'), is_cancel=0)

In [12]:
gaps = []

# Iterate through each entry
for tup in cust.itertuples():
    i = tup[0]
    expire_date = tup[-2]
    if i == len(cust) - 1:
        gaps.append(np.nan)
    else:
        next_trans = cust.loc[i+1, :].copy()
        j = 0
        while next_trans['is_cancel'] == 1:
            next_trans = cust.loc[i + j, :].copy()
            j += 1
            # Handle the case where the last transaction is cancelled
            if i + j >= len(cust):
                gaps.append(np.nan)
            
        next_start_date = next_trans['transaction_date']
        gaps.append((next_start_date - expire_date).days)

In [13]:
cust['gap'] = gaps

This customer does not have any churns if we define churns as a period of 30 days without renewing. However, they would have one churn if measuring by for example, two weeks. 

The next step is to find out when the churn actually occurs. We'll use two weeks as the churn period so we actually have one churn.

In [14]:
# Days without membership to be considered a churn
periods = 14

# Determine if churned occur
cust['churn'] = cust['gap'] > periods
cust['potential_churn_date'] = cust['membership_expire_date'] + pd.Timedelta(periods, unit = 'd')

# If customer did churn set the churn date
cust.loc[cust['churn'] == 1, 'churn_date'] = cust.loc[cust['churn'] == 1, 'potential_churn_date']
cust.head()
cust.tail()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,gap,churn,potential_churn_date,churn_date
0,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-01-13,2016-12-15,0,-694.0,False,2016-12-29,NaT
1,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,119,1,2015-01-21,2017-01-15,0,-720.0,False,2017-01-29,NaT
2,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-01-26,2017-02-15,0,-733.0,False,2017-03-01,NaT
3,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-02-13,2017-03-15,0,-753.0,False,2017-03-29,NaT
4,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,119,1,2015-02-21,2017-04-12,0,-776.0,False,2017-04-26,NaT


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,gap,churn,potential_churn_date,churn_date
46,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,129,129,1,2016-11-14,2016-11-14,1,0.0,False,2016-11-28,NaT
47,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2016-11-14,2016-12-13,0,0.0,False,2016-12-27,NaT
48,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2016-12-13,2017-01-13,0,0.0,False,2017-01-27,NaT
49,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2017-01-13,2017-02-13,0,28.0,True,2017-02-27,2017-02-27
50,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2017-03-13,2017-04-13,0,NaN,False,2017-04-27,NaT


# Make Labels

The actual labels are going to occur either on a monthly or two-week basis. For the labels, at the `cutoff_time`, we'll ask two questions: does the customer churn in the next [two weeks / month] and how long until the customer churns.

## Month Labels

We'll start by making labels for each month that the customer was a member.

In [15]:
first_trans = cust['transaction_date'].min()
last_trans = cust['membership_expire_date'].max()
start_date = pd.datetime(first_trans.year, first_trans.month, 1)
end_date = pd.datetime(last_trans.year, last_trans.month, 1)

# Create a range of months
date_range = pd.date_range(start_date, end_date, freq = 'MS')
date_range[:5]
date_range[-5:]

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01'],
              dtype='datetime64[ns]', freq='MS')

DatetimeIndex(['2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
               '2019-10-01'],
              dtype='datetime64[ns]', freq='MS')

In [16]:
labels = pd.DataFrame({'cutoff_time': date_range})
labels['next_cutoff_time'] = labels['cutoff_time'].shift(-1)

previous_churn = None

# Iterate through the churn dates
for churn_date in cust.loc[cust['churn_date'].notnull(), 'churn_date']:
    print(churn_date)
    # Assign the label 1 if the customer churned during the cutoff_time period
    labels.loc[(labels['cutoff_time'] <= churn_date) & (labels['next_cutoff_time'] > churn_date), 'churn'] = 1
    
    # If there was a previous churn
    if previous_churn is not None:
        # Subset to cutoff times after the previous churn but before the current churn
        # Calculate the days until the churn
        labels.loc[(labels['cutoff_time'] > previous_churn) & 
                   (labels['cutoff_time'] <= churn_date), 
                   'days_to_next_churn'] = (churn_date - labels.loc[(labels['cutoff_time'] > previous_churn) & 
                                                                   (labels['cutoff_time'] <= churn_date), 'cutoff_time']).dt.days
    # No previous churn
    else:
        # Subset to cutoff times before the current churn and calculate days until the churn
        labels.loc[labels['cutoff_time'] <= churn_date, 
                   'days_to_next_churn'] = (churn_date - labels.loc[labels['cutoff_time'] <= churn_date,
                                                                     'cutoff_time']).dt.days
    previous_churn = churn_date

2017-02-27 00:00:00


In [17]:
labels[labels['churn'] == 1]

,cutoff_time,next_cutoff_time,churn,days_to_next_churn
25,2017-02-01,2017-03-01,1.0,26.0


Those are the month labels. Let's write a function that calculates the month labels for one customer.

In [18]:
def generate_month_labels(customer_id, trans, churn_period = 30, return_cust = False):
    """Make labels for one customer for one month
    Params
    --------
        customer_id (str): string used to select customer
        churn_period (int): number of days without membership required for a churn
        
    Return
    --------
        labels (dataframe): labels for all months in customer history
    """
    cust = trans.loc[trans['msno'] == customer_id].copy()
    
    # Make sure to sort transactions
    cust = cust.sort_values(['transaction_date', 'membership_expire_date']).reset_index(drop = True)
    gaps = []

    # Iterate through each entry
    for tup in cust.itertuples():
        i = tup[0]
        expire_date = tup[8]
        if i == len(cust) - 1:
            gaps.append(np.nan)
        else:
            next_trans = cust.loc[i+1, :].copy()
            j = 0
            last_cancelled = False
            while next_trans['is_cancel'] == 1:
                # Handle the case where the last transaction is cancelled
                if i + j >= len(cust):
                    last_cancelled = True
                    gaps.append(np.nan)
                else:
                    next_trans = cust.loc[i + j, :].copy()
                    j += 1
                

            next_start_date = next_trans['transaction_date']
            if not last_cancelled:
                gaps.append((next_start_date - expire_date).days)

    cust['gap'] = gaps
    
    # Determine if churned occur
    cust['churn'] = cust['gap'] > churn_period
    
    # Calculate date range for labels
    first_trans = cust['transaction_date'].min()
    last_trans = cust['membership_expire_date'].max()
    start_date = pd.datetime(first_trans.year, first_trans.month, 1)
    if last_trans.month == 12:
        end_date = pd.datetime(last_trans.year + 1, 1, 1)
    else:
        end_date = pd.datetime(last_trans.year, last_trans.month + 1, 1)

    # Create a range of month starts
    date_range = pd.date_range(start_date, end_date, freq = 'MS')
    
    # Create a label dataframe
    labels = pd.DataFrame({'cutoff_time': date_range})
    labels['next_cutoff_time'] = labels['cutoff_time'].shift(-1)
    labels['msno'] = customer_id
    
    # Handle case where there are no churns
    if not np.any(cust['churn'] == 1):
        labels['churn'] = 0
        labels['days_to_next_churn'] = np.nan
        return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]
    
    # If customer did churn set the churn date
    cust['potential_churn_date'] = cust['membership_expire_date'] + pd.Timedelta(churn_period, unit = 'd')
    cust.loc[cust['churn'] == 1, 'churn_date'] = cust.loc[cust['churn'] == 1, 'potential_churn_date']
    
    previous_churn = None

    # Iterate through the churn dates
    for churn_date in cust.loc[cust['churn_date'].notnull(), 'churn_date']:
        # Assign the label 1 if the customer churned during the cutoff_time period
        labels.loc[(labels['cutoff_time'] <= churn_date) & (labels['next_cutoff_time'] > churn_date), 'churn'] = 1

        # If there was a previous churn
        if previous_churn is not None:
            # Subset to cutoff times after the previous churn but before the current churn
            # Calculate the days until the churn
            labels.loc[(labels['cutoff_time'] > previous_churn) & 
                       (labels['cutoff_time'] <= churn_date), 
                       'days_to_next_churn'] = (churn_date - labels.loc[(labels['cutoff_time'] > previous_churn) & 
                                                                       (labels['cutoff_time'] <= churn_date), 'cutoff_time']).dt.days
        # No previous churn
        else:
            # Subset to cutoff times before the current churn and calculate days until the churn
            labels.loc[labels['cutoff_time'] <= churn_date, 
                       'days_to_next_churn'] = (churn_date - labels.loc[labels['cutoff_time'] <= churn_date,
                                                                         'cutoff_time']).dt.days
        previous_churn = churn_date
        
    labels['churn'] = labels['churn'].fillna(0)
    
    if return_cust:
        return cust, labels
    
    return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]

In [19]:
cutoff_times = generate_month_labels('/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=', trans, 14)
cutoff_times[cutoff_times['churn'] == 1].head()

,msno,cutoff_time,churn,days_to_next_churn
25,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,2017-02-01,1.0,26.0


In [20]:
cutoff_times = generate_month_labels('/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=',  trans, 30)
cutoff_times[cutoff_times['churn'] == 1].head()

,msno,cutoff_time,churn,days_to_next_churn


In [21]:
cutoff_times = generate_month_labels('5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=', trans)
cutoff_times.head()

,msno,cutoff_time,churn,days_to_next_churn
0,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-01-01,0,NaN
1,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-02-01,0,NaN
2,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-03-01,0,NaN
3,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-04-01,0,NaN
4,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-05-01,0,NaN


In [23]:
cutoff_list = []

# Iterate through every customer
for customer_id in trans['msno'].unique():
    cutoff_list.append(generate_month_labels(customer_id, trans, 30))
    
cutoff_times = pd.concat(cutoff_list)
cutoff_times.head()
cutoff_times.tail()

,msno,cutoff_time,churn,days_to_next_churn
0,+/6nRSzfF+CIynhnBM5xz8J6ArlSdLY74gsNK09dbes=,2015-06-01,0.0,NaN
1,+/6nRSzfF+CIynhnBM5xz8J6ArlSdLY74gsNK09dbes=,2015-07-01,0.0,NaN
0,+1TKL6EWVDuKFAOvWZOsoGTILy2POMnxxvUgP7PPCy8=,2016-11-01,0.0,NaN
1,+1TKL6EWVDuKFAOvWZOsoGTILy2POMnxxvUgP7PPCy8=,2016-12-01,0.0,NaN
0,+58aOzMPOZSi0END5IUKzK009k/iGY9mB+k9s5qetAI=,2016-11-01,0.0,NaN


,msno,cutoff_time,churn,days_to_next_churn
24,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,2017-01-01,0.0,NaN
25,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,2017-02-01,0.0,NaN
26,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,2017-03-01,0.0,NaN
27,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,2017-04-01,0.0,NaN
28,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,2017-05-01,0.0,NaN


In [24]:
cust, c = generate_month_labels('0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=', trans, return_cust = True)

In [25]:
cust, c = generate_month_labels('+6UN6VJD8u9vZm4lZRAREpzBRM4YoeOSWhEX0c5JBAU=', trans, return_cust = True)

# All Labels for Partition 

The next step is to write a function that can make the month labels for a partition. It will accept a partition number and generate the partition labels. These will then be saved to the partition directory.

In [26]:
partitions = list(range(len(os.listdir('/data/churn/partitions/'))))
partitions[-1]

999

In [27]:
def partition_to_labels(partition, label_type = 'month'):
    trans_file = f'{base_dir}{partition}/transactions.csv'
    print(trans_file)
    trans = pd.read_csv(f'{base_dir}p{partition}/transactions.csv',
                       parse_dates=['transaction_date', 'membership_expire_date'], 
                        infer_datetime_format = True)

    trans = trans.loc[trans['membership_expire_date'] >= trans['transaction_date']]
    cutoff_list = []

    if label_type == 'month':
        # Iterate through every customer
        for customer_id in trans['msno'].unique():
            cutoff_list.append(generate_month_labels(customer_id, trans, 30))
        cutoff_times = pd.concat(cutoff_list)
        cutoff_times.to_csv(f'{base_dir}p{partition}/month_labels.csv', index = False)
        
        

In [28]:
partition_to_labels(1)

/data/churn/partitions/1/transactions.csv


KeyboardInterrupt: 

In [ ]:
partition_to_labels(50)

# Two Week Labels

Next we'll work on making labels in two-week increments. This time, the question is not whether the customer churned during the subsequent month, but whether the customer churned in the two weeks following the `cutoff_time`.

In [29]:
def generate_biweekly_labels(customer_id, trans, churn_period = 30, return_cust = False):
    """Make labels for one customer for one month
    Params
    --------
        customer_id (str): string used to select customer
        churn_period (int): number of days without membership required for a churn
        
    Return
    --------
        labels (dataframe): labels for all months in customer history
    """
    cust = trans.loc[trans['msno'] == customer_id].copy()
    
    # Make sure to sort transactions
    cust = cust.sort_values(['transaction_date', 'membership_expire_date']).reset_index(drop = True)
    gaps = []

    # Iterate through each entry
    for tup in cust.itertuples():
        i = tup[0]
        expire_date = tup[8]
        if i == len(cust) - 1:
            gaps.append(np.nan)
        else:
            next_trans = cust.loc[i+1, :].copy()
            j = 0
            last_cancelled = False
            while next_trans['is_cancel'] == 1:
                # Handle the case where the last transaction is cancelled
                if i + j >= len(cust):
                    last_cancelled = True
                    gaps.append(np.nan)
                else:
                    next_trans = cust.loc[i + j, :].copy()
                    j += 1
                

            next_start_date = next_trans['transaction_date']
            if not last_cancelled:
                gaps.append((next_start_date - expire_date).days)

    cust['gap'] = gaps
    
    # Determine if churned occur
    cust['churn'] = cust['gap'] > churn_period
    
    # Calculate date range for labels
    first_trans = cust['transaction_date'].min()
    last_trans = cust['membership_expire_date'].max()
    start_date = pd.datetime(first_trans.year, first_trans.month, 1)
    if last_trans.month == 12:
        end_date = pd.datetime(last_trans.year + 1, 1, 1)
    else:
        end_date = pd.datetime(last_trans.year, last_trans.month + 1, 1)

    # Create a range of month starts
    date_range = pd.date_range(start_date, end_date, freq = 'SM')
    
    # Create a label dataframe
    labels = pd.DataFrame({'cutoff_time': date_range})
    labels['next_cutoff_time'] = labels['cutoff_time'].shift(-1)
    labels['msno'] = customer_id
    
    # Handle case where there are no churns
    if not np.any(cust['churn'] == 1):
        labels['churn'] = 0
        labels['days_to_next_churn'] = np.nan
        return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]
    
    # If customer did churn set the churn date
    cust['potential_churn_date'] = cust['membership_expire_date'] + pd.Timedelta(churn_period, unit = 'd')
    cust.loc[cust['churn'] == 1, 'churn_date'] = cust.loc[cust['churn'] == 1, 'potential_churn_date']
    
    previous_churn = None

    # Iterate through the churn dates
    for churn_date in cust.loc[cust['churn_date'].notnull(), 'churn_date']:
        # Assign the label 1 if the customer churned during the cutoff_time period
        labels.loc[(labels['cutoff_time'] <= churn_date) & (labels['next_cutoff_time'] > churn_date), 'churn'] = 1

        # If there was a previous churn
        if previous_churn is not None:
            # Subset to cutoff times after the previous churn but before the current churn
            # Calculate the days until the churn
            labels.loc[(labels['cutoff_time'] > previous_churn) & 
                       (labels['cutoff_time'] <= churn_date), 
                       'days_to_next_churn'] = (churn_date - labels.loc[(labels['cutoff_time'] > previous_churn) & 
                                                                       (labels['cutoff_time'] <= churn_date), 'cutoff_time']).dt.days
        # No previous churn
        else:
            # Subset to cutoff times before the current churn and calculate days until the churn
            labels.loc[labels['cutoff_time'] <= churn_date, 
                       'days_to_next_churn'] = (churn_date - labels.loc[labels['cutoff_time'] <= churn_date,
                                                                         'cutoff_time']).dt.days
        previous_churn = churn_date
        
    labels['churn'] = labels['churn'].fillna(0)
    
    if return_cust:
        return cust, labels
    
    return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]

In [30]:
bw_labels = generate_biweekly_labels('5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=', trans)

In [31]:
bw_labels

,msno,cutoff_time,churn,days_to_next_churn
0,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-01-15,0,NaN
1,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-01-31,0,NaN
2,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-02-15,0,NaN
3,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-02-28,0,NaN
4,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-03-15,0,NaN
5,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-03-31,0,NaN
6,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-04-15,0,NaN
7,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-04-30,0,NaN
8,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-05-15,0,NaN
9,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-05-31,0,NaN


In [33]:
pd.options.display.max_rows = 200
bw_labels = generate_biweekly_labels('0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=', trans)
bw_labels

,msno,cutoff_time,churn,days_to_next_churn
0,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-01-15,0.0,530.0
1,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-01-31,0.0,514.0
2,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-02-15,0.0,499.0
3,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-02-28,0.0,486.0
4,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-03-15,0.0,471.0
5,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-03-31,0.0,455.0
6,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-04-15,0.0,440.0
7,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-04-30,0.0,425.0
8,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-05-15,0.0,410.0
9,0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=,2015-05-31,0.0,394.0


# Use Dask to Parallelize Making Cutoff Times

In [ ]:
partitions = list(range(len(os.listdir('/data/churn/partitions'))))
len(partitions)

In [ ]:
import dask.bag as db
from dask.distributed import Client

# Use all cores
client = Client(processes = True)

In [ ]:
client.ncores()

In [ ]:
# Create a bag object
b = db.from_sequence(partitions, npartitions=len(partitions))

# Map partition making function
b = b.map(make_partition_labels)
    
b

In [ ]:
%%capture
from timeit import default_timer as timer

start = timer()
b.compute()
end = timer()



In [ ]:
print(f'{round(end - start)} seconds elapsed.')

# Conclusions

In this notebook, we generated labels and cutoff time dataframes for each partition. We can then use these cutoff times in a call to deep feature synthesis to make features for each label. 

The next notebook is `Feature Engineering`. 